In [40]:
%pip install eep153_tools
%pip install python_gnupg
%pip install -U gspread_pandas
#load in file from class
def format_id(id,zeropadding=0):
    """Nice string format for any id, string or numeric.

    Optional zeropadding parameter takes an integer
    formats as {id:0z} where
    """
    if pd.isnull(id) or id in ['','.']: return None

    try:  # If numeric, return as string int
        return ('%d' % id).zfill(zeropadding)
    except TypeError:  # Not numeric
        return id.split('.')[0].strip().zfill(zeropadding)
    except ValueError:
        return None

data_url = "https://docs.google.com/spreadsheets/d/12Z4n8HbFZRYvH6B-D8EDLDibRiL50zNMlSBLMJ41C1o/"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [41]:
import pandas as pd
import re 
from eep153_tools.sheets import read_sheets

#create recipes df
recipes = read_sheets(data_url, sheet="recipes")
recipes = (recipes
           .assign(parent_foodcode = lambda df: df["parent_foodcode"].apply(format_id),
                   ingred_code = lambda df: df["ingred_code"].apply(format_id))
           .rename(columns={"parent_desc": "recipe"}))
recipes.head()
#recipes.shape


,parent_foodcode,recipe,ingred_code,ingred_desc,ingred_wt
0,11340000,"Imitation milk, non-soy, sweetened",43543,"Milk, imitation, non-soy",100.0
1,11460150,"Yogurt, frozen, NS as to flavor, lowfat milk",1298,"Yogurt, frozen, flavors other than chocolate, ...",100.0
2,11460160,"Yogurt, frozen, chocolate, lowfat milk",1117,"Yogurt, plain, low fat, 12 grams protein per 8...",81.8
3,11460160,"Yogurt, frozen, chocolate, lowfat milk",19166,"Cocoa, dry powder, unsweetened, processed with...",5.2
4,11460160,"Yogurt, frozen, chocolate, lowfat milk",19335,"Sugars, granulated",13.0


In [42]:
#check the data types
recipes.dtypes

parent_foodcode     object
recipe              object
ingred_code         object
ingred_desc         object
ingred_wt          float64
dtype: object

In [43]:
#List of non-vegan keywords
NON_VEGAN_KEYWORDS = [
    "beef", "pork", "chicken", "turkey", "fish", "seafood", "shellfish", "shrimp", "crab","crabs",
    "lamb", "goat", "duck", "goose", "tuna", "salmon", "cod", "bacon", "ham",
    "shellfish", "lobster", "mussels", "oysters", "scallops", "octopus", "eel",
    "organ meat", "milk","Eggnog" "cheese", "butter", "cream","ice cream", "yogurt", "whey",
    "casein", "lactose", "ghee", "buttermilk", "egg", "eggs", "mayo", "mayonnaise", "albumen",
    "albumin", "lysozyme", "ovomucoid", "ovomucin", "ovovitellin", "honey",
    "bee pollen", "royal jelly", "propolis", "shellac", "confectioner’s glaze",
    "carmine", "cochineal", "lard", "tallow", "suet", "gelatin", "collagen",
    "isinglass", "bone broth", "bone stock", "fish sauce", "oyster sauce",
    "shrimp paste", "worcestershire sauce", "anchovies", "rennet", "pepsin",
    "bone char", "vitamin d3", "lanolin", "omega-3 fish oil", "caseinate",
    "lecithin (egg)", "cysteine", "l-cysteine", "glycerin (animal)",
    "glycerol (animal)", "stearic acid (animal)", "tallowate", "sodium tallowate",
    "capric acid", "caprylic acid"
]

#this partal match: "milkshake" or "eggroll" will get flagged (since "milk" or "egg" is in the keyword list).
NON_VEGAN_PATTERN = re.compile(
    '|'.join(map(re.escape, NON_VEGAN_KEYWORDS)),
    re.IGNORECASE
)


def filter_vegan_ingredients(df: pd.DataFrame) -> pd.DataFrame:
    
    df[["recipe", "ingred_desc"]] = df[["recipe", "ingred_desc"]].astype(str)

    
    mask = ~(df["recipe"].str.contains(NON_VEGAN_PATTERN, na=False, regex=True) |
             df["ingred_desc"].str.contains(NON_VEGAN_PATTERN, na=False, regex=True))
    
    return df[mask]

In [44]:
vegan_recipes = filter_vegan_ingredients(recipes)
vegan_recipes

,parent_foodcode,recipe,ingred_code,ingred_desc,ingred_wt
32,11830100,"Hot chocolate / Cocoa, dry mix, not reconstituted",14192,"Beverages, Cocoa mix, powder",100.0
33,11830115,"Hot chocolate / Cocoa, dry mix, no sugar added...",14196,"Beverages, Cocoa mix, no sugar added, powder",100.0
179,13170000,Baked Alaska,2047,"Salt, table",0.6
180,13170000,Baked Alaska,4610,"Margarine, regular, 80% fat, composite, stick,...",42.2
181,13170000,Baked Alaska,14411,"Water, tap, drinking",0.3
...,...,...,...,...,...
36035,99998210,Industrial oil as ingredient in food,4698,"Oil, industrial, canola, high oleic",25.0
36036,99998210,Industrial oil as ingredient in food,4699,"Oil, industrial, soy, low linolenic",25.0
36037,99998210,Industrial oil as ingredient in food,4700,"Oil, industrial, soy, ultra low linolenic",25.0
36038,99998210,Industrial oil as ingredient in food,4701,"Oil, industrial, soy, fully hydrogenated",25.0


In [45]:
#my idea is to filter over only foods that athelecs would eat. Organic, low fat etc.